In [140]:
import CCACSP
from preprocessing import *
from import_data import *
data_dir = 'DataCollection/outputs/'
ssvep_dir = data_dir + 'SSVEP/'
mi_dir = data_dir + 'MI/sess2/'
bci_dir1 = '../Documents/OpenBCI_GUI/Recordings/OpenBCISession_2023-02-28_12-03-49/'
bci_dir2 = '../Documents/OpenBCI_GUI/Recordings/OpenBCISession_2023-03-10_14-09-51/'
col_names = ['Fp1', 'O1', 'O2','C3','C4','P3','P4','Pz']
mi_files = get_files(mi_dir)
#mi_files = [mi_files[i] for i in range(len(mi_files)) if (i+1)%6!=0]
bci1_files = get_files(bci_dir1)
bci2_files = get_files(bci_dir2)

In [141]:
mi_dir_joelle = data_dir + 'MI/sess5/'
joelle_files = get_files(mi_dir_joelle)
joelle_files

['DataCollection/outputs/MI/sess5/2023-03-14T12:58:12.715082_MI_MIRF.txt',
 'DataCollection/outputs/MI/sess5/2023-03-14T12:58:12.715082_MI_NOMI.txt',
 'DataCollection/outputs/MI/sess5/2023-03-14T12:58:12.715082_MI_baseline.txt',
 'DataCollection/outputs/MI/sess5/2023-03-14T12:58:12.715082_MI_metadata.txt',
 'DataCollection/outputs/MI/sess5/2023-03-14T13:00:14.841521_MI_MILF.txt',
 'DataCollection/outputs/MI/sess5/2023-03-14T13:00:14.841521_MI_MIRF.txt',
 'DataCollection/outputs/MI/sess5/2023-03-14T13:00:14.841521_MI_MITw.txt',
 'DataCollection/outputs/MI/sess5/2023-03-14T13:00:14.841521_MI_MIfl.txt',
 'DataCollection/outputs/MI/sess5/2023-03-14T13:00:14.841521_MI_NOMI.txt',
 'DataCollection/outputs/MI/sess5/2023-03-14T13:00:14.841521_MI_baseline.txt',
 'DataCollection/outputs/MI/sess5/2023-03-14T13:00:14.841521_MI_metadata.txt']

In [142]:
from sklearn.model_selection import train_test_split

In [4]:
for i, fi in enumerate(joelle_files):
    print("{}: {}".format(i,fi))

0: DataCollection/outputs/MI/sess5/2023-03-14T12:58:12.715082_MI_MIRF.txt
1: DataCollection/outputs/MI/sess5/2023-03-14T12:58:12.715082_MI_NOMI.txt
2: DataCollection/outputs/MI/sess5/2023-03-14T12:58:12.715082_MI_baseline.txt
3: DataCollection/outputs/MI/sess5/2023-03-14T12:58:12.715082_MI_metadata.txt
4: DataCollection/outputs/MI/sess5/2023-03-14T13:00:14.841521_MI_MILF.txt
5: DataCollection/outputs/MI/sess5/2023-03-14T13:00:14.841521_MI_MIRF.txt
6: DataCollection/outputs/MI/sess5/2023-03-14T13:00:14.841521_MI_MITw.txt
7: DataCollection/outputs/MI/sess5/2023-03-14T13:00:14.841521_MI_MIfl.txt
8: DataCollection/outputs/MI/sess5/2023-03-14T13:00:14.841521_MI_NOMI.txt
9: DataCollection/outputs/MI/sess5/2023-03-14T13:00:14.841521_MI_baseline.txt
10: DataCollection/outputs/MI/sess5/2023-03-14T13:00:14.841521_MI_metadata.txt


In [5]:
bl_df = read_file(joelle_files[9], col_names)

In [6]:
def snip_eeg(eeg_df, Fs, snip_len, total_len):
    npts = total_len * Fs
    num_trials = int(len(list(eeg_df['Fp1']))/npts)
    snip_df = pd.DataFrame()
    for chan in eeg_df.columns:
        chan_list = []
        for i in range(num_trials):
            chan_trial = list(eeg_df[chan])[npts*i:npts*i+int((total_len-snip_len)*Fs)]
            chan_list += chan_trial
        snip_df[chan] = chan_list
    return snip_df

In [79]:
Fs = 250
def get_train_data(mi_file, col_names, baseline_file, snip_len=0):
    #eeg, Fs = process_eeg(mi_file, col_names, bci_file, low=2)
    eeg = read_file(mi_file, col_names)
    baseline = read_file(baseline_file, col_names)
    bc = baseline_correct(eeg, baseline)
    snip = snip_eeg(bc,Fs,snip_len,5)
    filt_eeg = filter_eeg(snip,Fs,1,100)
    eeg_np = filt_eeg.to_numpy()
    npts = int((5-snip_len)*Fs)
    num_trials = int(eeg_np.shape[0] / npts)
    train_data = np.array([eeg_np[i*npts:(i+1)*npts].reshape(8,npts) for i in range(num_trials)])
    return train_data

In [82]:
twiz = get_train_data(joelle_files[6], col_names, joelle_files[9],2)

In [83]:
twiz.shape

(4, 8, 750)

In [139]:
def get_session_data():
    data = []
    #for fi in mi_files[5*sess_num:5*(sess_num+1)-1]:
    for fi in joelle_files[4:8]:
        print(fi)
        data.append(get_train_data(fi, col_names, joelle_files[9]))
    class1 = np.concatenate(data)
    class2 = get_train_data(joelle_files[8], col_names, joelle_files[9])
    minLen = min(len(class1),len(class2))
    return class1[:minLen], class2[:minLen]

In [131]:
def get_all_data(num_sessions, bci_file):
    mi = []
    nomi = []
    for i in range(num_sessions):
        class1, class2 = get_session_data(i, bci_file)
        mi.append(class1)
        nomi.append(class2)
    mi = np.concatenate(mi)
    nomi = np.concatenate(nomi)
    minLen = min(len(mi), len(nomi))
    mi = mi[:minLen]
    nomi = nomi[:minLen]
    return mi, nomi 

In [132]:
mi, nomi = get_session_data()

DataCollection/outputs/MI/sess5/2023-03-14T13:00:14.841521_MI_MILF.txt
DataCollection/outputs/MI/sess5/2023-03-14T13:00:14.841521_MI_MIRF.txt
DataCollection/outputs/MI/sess5/2023-03-14T13:00:14.841521_MI_MITw.txt
DataCollection/outputs/MI/sess5/2023-03-14T13:00:14.841521_MI_MIfl.txt


In [133]:
mi.shape, nomi.shape

((16, 8, 1187), (16, 8, 1187))

In [134]:
def get_train_test(mi, nomi, split=0.8):
    np.random.shuffle(mi)
    np.random.shuffle(nomi)
    splitIdx = int(split*len(mi))
    train_mi = mi[:splitIdx]
    test_mi = mi[splitIdx:]
    train_nomi = nomi[:splitIdx]
    test_nomi = nomi[splitIdx:]
    return train_mi, train_nomi, test_mi, test_nomi

In [135]:
def get_results(train_mi, train_nomi, test_mi, test_nomi,show_results=True):
    numFilt = 8
    filters, clf = CCACSP.train(train_mi, train_nomi, numFilt)
    mi_res = []
    nomi_res = []
    for i in range(len(test_mi)):
        mi_result = CCACSP.test(test_mi[i], filters, clf)
        mi_res.append(mi_result[0])
        #print("True class: MI, Predicted: {}".format(mi_result[0]))
        nomi_result = CCACSP.test(test_nomi[i], filters, clf)
        nomi_res.append(nomi_result[0])
        #print("True class: NOMI, Predicted: {}".format(nomi_result[0]))
    decode = {0: 'MI', 1:'NOMI'}
    mi_print = [decode[res] for res in mi_res]
    nomi_print = [decode[res] for res in nomi_res]
    acc = (np.sum([1-mi for mi in mi_res]) + np.sum([nomi for nomi in nomi_res])) / (len(mi_res) + len(nomi_res))
    if show_results:
        print('MI results: ', mi_print)
        print('NOMI results: ', nomi_print)
        print('Accuracy: ', acc)
    return acc

In [136]:
split = 0.6
train_mi, train_nomi, test_mi, test_nomi = get_train_test(mi, nomi, split=split)
get_results(train_mi, train_nomi, test_mi, test_nomi)

MI results:  ['NOMI', 'NOMI', 'MI', 'NOMI', 'MI', 'NOMI', 'MI']
NOMI results:  ['NOMI', 'MI', 'MI', 'NOMI', 'NOMI', 'NOMI', 'NOMI']
Accuracy:  0.5714285714285714


0.5714285714285714

In [137]:
def get_average_results(mi, nomi, split, num_tests):
    accs = []
    for i in range(num_tests):
        train_mi, train_nomi, test_mi, test_nomi = get_train_test(mi, nomi, split)
        accs.append(get_results(train_mi, train_nomi, test_mi, test_nomi, False))
    print('Average accuracy: ', np.mean(accs))

In [138]:
get_average_results(mi, nomi, split=0.8, num_tests=100)

Average accuracy:  0.6075


In [110]:
numFilt = 8
filers, clf = CCACSP.train(mi, nomi, numFilt)

In [120]:
mi_test, nomi_test = get_all_data(1, bci2_files[1])

DataCollection/outputs/MI/sess2/2023-03-10T14:51:16.448841_MI_MILF.txt
DataCollection/outputs/MI/sess2/2023-03-10T14:51:16.448841_MI_MIRF.txt
DataCollection/outputs/MI/sess2/2023-03-10T14:51:16.448841_MI_MISw.txt
DataCollection/outputs/MI/sess2/2023-03-10T14:51:16.448841_MI_MITy.txt


In [121]:
nomi_test = nomi_test[:12]

In [122]:
mi_test.shape, nomi_test.shape

((12, 8, 1250), (12, 8, 1250))

In [123]:
mi_res = []
nomi_res = []
for i in range(mi_test.shape[0]):
    mi_result = CCACSP.test(mi_test[i], filers, clf)
    mi_res.append(mi_result)
    print("True class: MI, Predicted: {}".format(mi_result[0]))
    nomi_result = CCACSP.test(nomi_test[i], filers, clf)
    nomi_res.append(nomi_result)
    print("True class: NOMI, Predicted: {}".format(nomi_result[0]))

True class: MI, Predicted: 0.0
True class: NOMI, Predicted: 1.0
True class: MI, Predicted: 0.0
True class: NOMI, Predicted: 1.0
True class: MI, Predicted: 0.0
True class: NOMI, Predicted: 1.0
True class: MI, Predicted: 0.0
True class: NOMI, Predicted: 1.0
True class: MI, Predicted: 0.0
True class: NOMI, Predicted: 1.0
True class: MI, Predicted: 0.0
True class: NOMI, Predicted: 1.0
True class: MI, Predicted: 0.0
True class: NOMI, Predicted: 1.0
True class: MI, Predicted: 0.0
True class: NOMI, Predicted: 1.0
True class: MI, Predicted: 0.0
True class: NOMI, Predicted: 1.0
True class: MI, Predicted: 1.0
True class: NOMI, Predicted: 1.0
True class: MI, Predicted: 0.0
True class: NOMI, Predicted: 1.0
True class: MI, Predicted: 1.0
True class: NOMI, Predicted: 1.0


In [124]:
np.mean(mi_res)

0.16666666666666666

In [125]:
np.mean(nomi_res)

1.0